In [ ]:
!pip install --pre deepchem rdkit-pypi

In [ ]:
!pip install --force-reinstall rdkit-pypi Pillow

In [ ]:
!pip install --pre deepchem rdkit-pypi scikit-learn matplotlib tensorflow Pillow # Or torch

In [ ]:
!pip install --upgrade pip
!pip install "numpy<2.0.0"  # Install NumPy 1.x FIRST
!pip install tf_keras # Install Keras 2 compatibility package


In [ ]:
!pip install tf_keras # Install Keras 2 compatibility package

# Set environment variable BEFORE importing TF/DeepChem
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import os
print(f"TF_USE_LEGACY_KERAS set to: {os.environ.get('TF_USE_LEGACY_KERAS')}")
# Expected output: TF_USE_LEGACY_KERAS set to: 1

TF_USE_LEGACY_KERAS set to: 1


In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import deepchem as dc
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem # RDKit for SMILES processing
from rdkit.Chem import Draw, Descriptors # For drawing molecules and calculating properties
from deepchem.metrics import Metric, roc_auc_score
from deepchem.models import GraphConvModel
from deepchem.feat import ConvMolFeaturizer # Used by 'GraphConv' featurizer string
from deepchem.splits import ScaffoldSplitter # Used by 'scaffold' splitter string
from deepchem.trans import BalancingTransformer # For handling imbalanced data
import collections # For counting class distribution
# PIL import is not strictly needed now but doesn't hurt
try:
    import PIL
except ImportError:
    print("Warning: PIL/Pillow library not found. Dependency check.")

# --- 1. Load and Featurize Data ---
print("Loading HIV dataset from MoleculeNet...")

# Use the 'GraphConv' featurizer (maps to ConvMolFeaturizer) suitable for GraphConvModel.
# Use 'scaffold' splitter for robust evaluation on chemical data.
# Use 'balancing' transformer to handle class imbalance.
tasks, datasets, transformers = dc.molnet.load_hiv(
    featurizer='GraphConv',
    splitter='scaffold',
    transformers=['balancing'] # Apply balancing transformation
)
train_dataset, valid_dataset, test_dataset = datasets

# Basic dataset information
n_tasks = len(tasks)
print(f"Number of tasks: {n_tasks}")
print(f"Task names: {tasks}") # Should be ['HIV_active']
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(valid_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print(f"Number of features: {train_dataset.X.shape}") # Shape of featurized data
print(f"Transformers applied: {[t.__class__.__name__ for t in transformers]}")


# --- 1.5 Visualize Dataset ---
print("\nVisualizing dataset characteristics...")

# Ensure we have data to visualize
if len(train_dataset) > 0:
    # 1.5.1 Class Distribution (using training set labels)
    try:
        labels = train_dataset.y.flatten() # Get the labels (0 or 1)
        label_counts = collections.Counter(labels)
        inactive_count = label_counts.get(0, 0)
        active_count = label_counts.get(1, 0)

        plt.figure(figsize=(6, 4))
        plt.bar(['Inactive (0)', 'Active (1)'], [inactive_count, active_count], color=['skyblue', 'salmon'])
        plt.title('Class Distribution in Training Set (HIV_active)')
        plt.ylabel('Number of Compounds')
        plt.text(0, inactive_count + 50, str(inactive_count), ha='center')
        plt.text(1, active_count + 50, str(active_count), ha='center')
        plt.tight_layout()
        plt.savefig("hiv_class_distribution.png")
        print("Class distribution plot saved to hiv_class_distribution.png")
        # plt.show() # Uncomment to display plot
        plt.close() # Close the plot to free memory

    except Exception as e:
        print(f"Could not generate class distribution plot: {e}")

    # 1.5.2 Molecular Weight Distribution (using training set SMILES)
    try:
        mol_weights = []
        invalid_smiles_count = 0
        # train_dataset.ids usually stores the SMILES strings
        smiles_list = train_dataset.ids
        for smiles in smiles_list:
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                mol_weights.append(Descriptors.MolWt(mol))
            else:
                invalid_smiles_count += 1

        if invalid_smiles_count > 0:
             print(f"Warning: Could not parse {invalid_smiles_count} SMILES for MolWt calculation.")

        if mol_weights:
            plt.figure(figsize=(8, 5))
            plt.hist(mol_weights, bins=50, color='lightblue', edgecolor='black')
            plt.title('Molecular Weight Distribution in Training Set')
            plt.xlabel('Molecular Weight (g/mol)')
            plt.ylabel('Frequency')
            plt.grid(axis='y', alpha=0.75)
            plt.tight_layout()
            plt.savefig("hiv_mol_weight_distribution.png")
            print("Molecular weight distribution plot saved to hiv_mol_weight_distribution.png")
            # plt.show() # Uncomment to display plot
            plt.close() # Close the plot

    except Exception as e:
        print(f"Could not generate molecular weight distribution plot: {e}")

    # 1.5.3 Draw Example Molecules (first active and inactive found)
    try:
        example_mols = []
        example_labels = []
        found_active = False
        found_inactive = False

        # Find one active and one inactive example with valid SMILES
        for i in range(len(train_dataset)):
            label = train_dataset.y[i][0]
            smiles = train_dataset.ids[i]
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                if label == 1 and not found_active:
                    example_mols.append(mol)
                    example_labels.append(f"Example Active (Idx {i})")
                    found_active = True
                elif label == 0 and not found_inactive:
                    example_mols.append(mol)
                    example_labels.append(f"Example Inactive (Idx {i})")
                    found_inactive = True
            if found_active and found_inactive:
                break # Stop once we have one of each

        if example_mols:
            # Generate the IPython Image object (common in notebooks)
            img = Draw.MolsToGridImage(
                example_mols,
                molsPerRow=2,
                subImgSize=(300, 300),
                legends=example_labels
            )

            # Check if image object was created
            if img:
                print(f"Type of object returned by MolsToGridImage: {type(img)}") # Diagnostic print
                # *** MODIFICATION START ***
                # Try to save using the .data attribute (for IPython.display.Image)
                if hasattr(img, 'data'):
                    print("Object has a 'data' attribute. Attempting to write data.")
                    # The data attribute should contain the PNG bytes
                    with open("hiv_example_molecules.png", "wb") as png_file:
                        png_file.write(img.data)
                    print("Example molecules image saved to hiv_example_molecules.png")
                else:
                    print("Object returned by MolsToGridImage lacks .data attribute. Cannot save image.")
                # *** MODIFICATION END ***
            else:
                 print("Could not create image object from molecules.")
        else:
            print("Could not find suitable example molecules to draw.")

    except Exception as e:
        print(f"Could not generate or save example molecules image: {e}")

else:
    print("Training dataset is empty, skipping visualization.")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Loading HIV dataset from MoleculeNet...
Number of tasks: 1
Task names: ['HIV_active']
Train dataset size: 32896
Validation dataset size: 4112
Test dataset size: 4112
Number of features: (32896,)
Transformers applied: ['BalancingTransformer']

Visualizing dataset characteristics...
Class distribution plot saved to hiv_class_distribution.png


[06:11:54] WARNING: not removing hydrogen atom without neighbors
[06:11:54] WARNING: not removing hydrogen atom without neighbors


Molecular weight distribution plot saved to hiv_mol_weight_distribution.png
Type of object returned by MolsToGridImage: <class 'IPython.core.display.Image'>
Object has a 'data' attribute. Attempting to write data.
Example molecules image saved to hiv_example_molecules.png


In [ ]:
# --- 2. Define and Train Model ---
print("\nDefining and training GraphConvModel...")

# Instantiate the Graph Convolutional Model
# n_tasks: Number of prediction tasks (1 for HIV activity)
# mode: 'classification' or 'regression'
# dropout: Regularization technique to prevent overfitting
# batch_size: Number of samples processed before the model is updated
# learning_rate: Step size for optimization algorithm
# batch_normalize: Whether to use batch normalization (Set to False to avoid 'fused' arg error)
# model_dir: Optional directory to save model checkpoints during training
model = GraphConvModel(
    n_tasks=n_tasks,
    mode='classification',
    batch_size=128,
    learning_rate=0.001,
    dropout=0.2,
    batch_normalize=False, # *** Set to False to avoid fused batch norm error ***
    # model_dir='./hiv_graphconv' # Uncomment to save checkpoints
)

# Define the evaluation metric: ROC AUC Score
metric = Metric(roc_auc_score, np.mean, mode='classification')

# Define a callback to evaluate on the validation set during training
# This helps monitor performance and can be used for early stopping
# It evaluates every `interval` steps.
callback = dc.models.ValidationCallback(
    valid_dataset,
    interval=500, # Evaluate every 500 training steps
    metrics=[metric],
    transformers=transformers, # Apply the same transformations used on training data
    # save_dir='./hiv_graphconv/best' # Uncomment to save the best performing model based on validation score
)

# Train the model
# nb_epoch: Number of passes through the entire training dataset
# callbacks: List of callbacks to apply during training
# deterministic: Set to True for reproducibility, False might be faster but less reproducible
num_epochs = 30 # Start with a smaller number (e.g., 10-30) for quicker testing
                # Increase (e.g., 50-100) for potentially better performance
print(f"Starting training for {num_epochs} epochs...")
loss = model.fit(
    train_dataset,
    nb_epoch=num_epochs,
    callbacks=[callback],
    deterministic=True # Set to True for reproducibility
)
print(f"Training completed. Final loss reported: {loss}")


# --- 3. Evaluate Model ---
print("\nEvaluating model performance on train, validation, and test sets...")

# Evaluate the trained model using the ROC AUC metric
train_scores = model.evaluate(train_dataset, [metric], transformers)
valid_scores = model.evaluate(valid_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

# Print the scores (mean-roc_auc_score is the key for the averaged score)
print(f"Train ROC AUC Score: {train_scores.get('mean-roc_auc_score', 'N/A'):.4f}")
print(f"Validation ROC AUC Score: {valid_scores.get('mean-roc_auc_score', 'N/A'):.4f}")
print(f"Test ROC AUC Score: {test_scores.get('mean-roc_auc_score', 'N/A'):.4f}")


Defining and training GraphConvModel...
Starting training for 30 epochs...
Step 500 validation: mean-roc_auc_score=0.681325
Step 1000 validation: mean-roc_auc_score=0.727801
Step 1500 validation: mean-roc_auc_score=0.689732
Step 2000 validation: mean-roc_auc_score=0.731687
Step 2500 validation: mean-roc_auc_score=0.722732
Step 3000 validation: mean-roc_auc_score=0.71317
Step 3500 validation: mean-roc_auc_score=0.75139
Step 4000 validation: mean-roc_auc_score=0.762915
Step 4500 validation: mean-roc_auc_score=0.770381
Step 5000 validation: mean-roc_auc_score=0.78039
Step 5500 validation: mean-roc_auc_score=0.794724
Step 6000 validation: mean-roc_auc_score=0.781569
Step 6500 validation: mean-roc_auc_score=0.777168
Step 7000 validation: mean-roc_auc_score=0.765797
Step 7500 validation: mean-roc_auc_score=0.789437
Training completed. Final loss reported: 1.104220199584961

Evaluating model performance on train, validation, and test sets...
Train ROC AUC Score: 0.8598
Validation ROC AUC Sco

In [ ]:
import json
# --- 4. Predict on New Candidate Molecules ---
print("\nPredicting activity for example candidate molecules...")

# Define some example SMILES strings
smiles_nevirapine = 'Clc1cccc(N2C(=O)c3c(nc(C4CC4)cn3)N(C)C2=O)c1'
smiles_lenacapavir = 'O=C(NC1=CC(=CC(F)=C1)S(=O)(=O)N1CCN(CC1)CC1=CC(F)=C(F)C=C1F)[C@H]1CCC[C@@H]1C1=C(C)N(C)N=C1C(C(=O)NC1=CC(F)=C(C(F)(F)F)C=C1)C1=NN=CN1CC1=C(Cl)C=CC=C1F'
smiles_ethanol = 'CCO'
smiles_candidate = 'O=C(O)c1cccc(C(=O)Nc2ccc(CN3CCN(C)CC3)cc2)c1'

candidate_smiles = [smiles_nevirapine, smiles_lenacapavir, smiles_ethanol, smiles_candidate]
candidate_names = ['Nevirapine', 'Lenacapavir', 'Ethanol', 'Candidate X']

# Store results for visualization
prediction_results = []

# Featurize the candidate molecules
print("Featurizing candidate molecules...")
featurizer = ConvMolFeaturizer() # Instantiate the featurizer explicitly for prediction
mols = [Chem.MolFromSmiles(s) for s in candidate_smiles]

# Handle potential RDKit errors (invalid SMILES)
valid_mols_indices = [i for i, m in enumerate(mols) if m is not None]
valid_mols = [mols[i] for i in valid_mols_indices]
valid_names = [candidate_names[i] for i in valid_mols_indices]
valid_smiles = [candidate_smiles[i] for i in valid_mols_indices]

if len(valid_mols) < len(mols):
    print("Warning: Some SMILES strings could not be parsed by RDKit and were skipped.")

if valid_mols:
    features = featurizer.featurize(valid_mols)
    prediction_dataset = dc.data.NumpyDataset(X=features)
    predictions_raw = model.predict(prediction_dataset)

    # Extract the probability of being 'Active' (Class 1 for task 0)
    if predictions_raw.ndim == 3 and predictions_raw.shape[2] == 2:
         active_probabilities = predictions_raw[:, 0, 1]
    elif predictions_raw.ndim == 2 and predictions_raw.shape[1] == 1:
         active_probabilities = predictions_raw[:, 0]
    else:
         print(f"Warning: Unexpected prediction output shape: {predictions_raw.shape}. Cannot reliably extract active probabilities.")
         active_probabilities = [np.nan] * len(valid_names)

    print("\nPredicted Activity Probabilities (Probability of being Active):")
    for name, prob in zip(valid_names, active_probabilities):
        print(f"- {name}: {prob:.4f}")
        # Store results for visualization
        prediction_results.append({'name': name, 'probability': float(prob)}) # Ensure float for JSON
else:
    print("No valid molecules provided for prediction.")


# --- 4.5 Generate Interactive Visualization ---
print("\nGenerating interactive prediction visualization...")

if prediction_results:
    # Create HTML content with embedded Plotly.js
    # Use json.dumps to safely embed Python data into JavaScript
    html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>HIV Candidate Prediction Results</title>
    <script src='https://cdn.plot.ly/plotly-latest.min.js'></script>
    <style>
        body {{ font-family: sans-serif; margin: 40px; }}
        h1 {{ text-align: center; color: #333; }}
        #plotDiv {{ margin-top: 30px; }}
    </style>
</head>
<body>
    <h1>Predicted HIV Activity Probabilities</h1>
    <div id='plotDiv'></div>

    <script>
        // Data passed from Python script
        const resultsData = {json.dumps(prediction_results)};

        // Extract names and probabilities for Plotly
        const names = resultsData.map(item => item.name);
        const probabilities = resultsData.map(item => item.probability);

        // Define the trace for the bar chart
        const trace1 = {{
            x: names,
            y: probabilities,
            type: 'bar',
            text: probabilities.map(p => p.toFixed(4)), // Text displayed on hover
            textposition: 'auto',
            hoverinfo: 'x+text', // Show name and probability text on hover
            marker: {{
                color: probabilities.map(p => p > 0.5 ? 'salmon' : 'skyblue'), // Color bars based on threshold
                line: {{
                    color: '#333',
                    width: 1
                }}
            }}
        }};

        // Define the layout for the chart
        const layout = {{
            title: 'Predicted Probability of Being Active against HIV',
            xaxis: {{
                title: 'Candidate Molecule'
            }},
            yaxis: {{
                title: 'Predicted Probability',
                range: [0, 1] // Ensure y-axis goes from 0 to 1
            }},
            bargap: 0.1 // Gap between bars
        }};

        // Data array for Plotly
        const data = [trace1];

        // Create the plot
        Plotly.newPlot('plotDiv', data, layout);
    </script>
</body>
</html>
"""
    # Write the HTML content to a file
    try:
        with open("hiv_predictions_visualization.html", "w", encoding="utf-8") as f:
            f.write(html_content)
        print("Interactive visualization saved to hiv_predictions_visualization.html")
    except Exception as e:
        print(f"Error saving HTML visualization: {e}")
else:
    print("No prediction results available to visualize.")


# --- 5. Visualize ROC Curve for Test Set ---
print("\nGenerating ROC curve for the test set...")

# Get predictions on the test set needed for the ROC curve
try:
    test_predictions_raw = model.predict(test_dataset)
    # Extract probabilities for the positive class (Active)
    if test_predictions_raw.ndim == 3 and test_predictions_raw.shape[2] == 2:
        test_probabilities_active = test_predictions_raw[:, 0, 1]
    elif test_predictions_raw.ndim == 2 and test_predictions_raw.shape[1] == 1:
        test_probabilities_active = test_predictions_raw[:, 0]
    else:
         raise ValueError(f"Unexpected prediction shape on test set: {test_predictions_raw.shape}")

    test_true_labels = test_dataset.y[:, 0] # True labels for the first task

    # Calculate ROC curve points using scikit-learn
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, thresholds = roc_curve(test_true_labels, test_probabilities_active)
    roc_auc_value = auc(fpr, tpr)

    # Plot the ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_value:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance level (AUC = 0.50)') # Random guess line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05]) # Add a little padding at the top
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('Receiver Operating Characteristic (ROC) Curve - HIV Test Set')
    plt.legend(loc="lower right")
    plt.grid(True)
    # Save the figure
    plt.savefig("hiv_prediction_roc_curve.png")
    print("ROC curve plot saved to hiv_prediction_roc_curve.png")
    # plt.show() # Uncomment to display the plot directly if running interactively
    plt.close() # Close the plot

except ImportError:
    print("\nError: Scikit-learn library not found. Cannot generate ROC curve plot.")
    print("Please install it: pip install scikit-learn")
except Exception as e:
    print(f"\nAn error occurred during ROC curve generation: {e}")

print("\n--- Process Finished ---")



Predicting activity for example candidate molecules...
Featurizing candidate molecules...

Predicted Activity Probabilities (Probability of being Active):
- Nevirapine: 0.2425
- Lenacapavir: 0.6606
- Ethanol: 0.0862
- Candidate X: 0.6627

Generating interactive prediction visualization...
Interactive visualization saved to hiv_predictions_visualization.html

Generating ROC curve for the test set...
ROC curve plot saved to hiv_prediction_roc_curve.png

--- Process Finished ---


#generative Modelling

In [ ]:
# Required libraries: deepchem, rdkit-pypi, tensorflow, numpy<2, tf_keras
# Install using pip in Colab (run in separate cell FIRST, then restart runtime):
# !pip install --upgrade pip
# !pip install "numpy<2.0.0"  # Install NumPy 1.x FIRST
# !pip install tf_keras # Install Keras 2 compatibility package
# import os; os.environ['TF_USE_LEGACY_KERAS'] = '1' # Set Env Var AFTER installing tf_keras
# !pip install --pre deepchem rdkit-pypi scikit-learn matplotlib tensorflow Pillow # Or torch

import deepchem as dc
import numpy as np
import tensorflow as tf
from tensorflow import keras
# Use Keras 2 API if using TF_USE_LEGACY_KERAS=1
# from tf_keras.layers import LSTM, Dense, Embedding, TimeDistributed
# from tf_keras.models import Sequential
# from tf_keras.preprocessing.sequence import pad_sequences
# from tf_keras.utils import to_categorical
# Or use standard Keras 3 if TF_USE_LEGACY_KERAS is not set or needed
from keras.layers import LSTM, Dense, Embedding, TimeDistributed
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from rdkit import Chem # For validating generated SMILES
import random

# --- Configuration ---
MAX_SMILES_LEN = 100 # Maximum length for SMILES strings (adjust based on dataset)
LSTM_UNITS = 128
EMBEDDING_DIM = 64
NUM_EPOCHS = 5 # Increase for better results, but takes longer
BATCH_SIZE = 128
NUM_TO_GENERATE = 10 # Number of molecules to attempt generating

# --- 1. Load SMILES Data ---
print("Loading HIV dataset SMILES...")
# Provide a featurizer (e.g., 'ECFP') as required by molnet loader,
# even though we only need the SMILES IDs. Splitter is not needed.
# *** MODIFICATION START ***
tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', splitter=None)
# *** MODIFICATION END ***

# datasets is a tuple, get the first (and only) dataset
dataset = datasets[0]
all_smiles = dataset.ids # .ids usually holds the SMILES strings

# Filter SMILES by length
original_count = len(all_smiles)
all_smiles = [s for s in all_smiles if isinstance(s, str) and len(s) <= MAX_SMILES_LEN] # Add check for string type
print(f"Loaded {original_count} SMILES, using {len(all_smiles)} with length <= {MAX_SMILES_LEN}")

# --- 2. Preprocess SMILES Data ---
print("Preprocessing SMILES data...")

# Add start (!) and end (?) tokens
smiles_with_tokens = ['!' + s + '?' for s in all_smiles]

# Create character vocabulary
all_chars = set()
for s in smiles_with_tokens:
    all_chars.update(s)
chars = sorted(list(all_chars))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}
vocab_size = len(chars)
print(f"Vocabulary size: {vocab_size}")
print(f"Vocabulary: {''.join(chars)}")

# Convert SMILES to sequences of integers
sequences = [[char_to_int[c] for c in s] for s in smiles_with_tokens]

# Prepare input (X) and output (y) sequences for training
# X: sequence up to character t
# y: sequence shifted by one (predict character t+1)
X_seq, y_seq = [], []
for seq in sequences:
    # Ensure sequence has at least 2 characters (start + one char)
    if len(seq) > 1:
      # Input sequence (excluding the last character)
      X_seq.append(seq[:-1])
      # Output sequence (excluding the first character)
      y_seq.append(seq[1:])

print(f"Number of sequences for training: {len(X_seq)}")

# Pad sequences to ensure uniform length
# Use max length including start/end tokens
max_len_with_tokens = MAX_SMILES_LEN + 2
X_padded = pad_sequences(X_seq, maxlen=max_len_with_tokens, padding='post')
y_padded = pad_sequences(y_seq, maxlen=max_len_with_tokens, padding='post')

# One-hot encode the output sequences (y)
# Shape: (num_sequences, max_len_with_tokens, vocab_size)
y_one_hot = np.array([to_categorical(seq, num_classes=vocab_size) for seq in y_padded])

print(f"Shape of X_padded: {X_padded.shape}")
print(f"Shape of y_one_hot: {y_one_hot.shape}")


# --- 3. Define LSTM Model ---
print("Defining LSTM model...")

model = Sequential([
    # Embedding layer: Turns integer indices into dense vectors
    Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=max_len_with_tokens),
    # LSTM layer: Processes sequences, returns full sequence output
    LSTM(LSTM_UNITS, return_sequences=True),
    # TimeDistributed Dense layer: Applies a Dense layer to each time step
    # Output layer predicts probability distribution over the vocabulary for the next character
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


# --- 4. Train LSTM Model ---
# Note: Training can be slow, especially without a GPU.
# Increase epochs for better generation quality.
print(f"\nTraining LSTM model for {NUM_EPOCHS} epochs...")
history = model.fit(X_padded, y_one_hot, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1)
print("Training finished.")




In [ ]:
model

In [ ]:
# --- 5. Generate New Molecules ---
print(f"\nGenerating {NUM_TO_GENERATE} new molecule attempts...")

def generate_smiles(model, char_to_int, int_to_char, max_len, start_token='!', end_token='?'):
    """Generates a SMILES string using the trained LSTM model."""
    vocab_size = len(char_to_int)
    generated_sequence = [char_to_int[start_token]]

    while True:
        # Pad the current sequence
        padded_sequence = pad_sequences([generated_sequence], maxlen=max_len, padding='post')

        # Predict probabilities for the next character
        # Predict gives shape (1, current_len, vocab_size), take last prediction
        probabilities = model.predict(padded_sequence, verbose=0)[0, len(generated_sequence)-1, :]

        # Sample the next character index based on probabilities
        # Using np.random.choice introduces randomness
        # Add small epsilon to probabilities to avoid sum-to-zero issues if model predicts all zeros
        probabilities = probabilities + 1e-8
        probabilities = probabilities / np.sum(probabilities) # Re-normalize

        next_char_index = np.random.choice(range(vocab_size), p=probabilities)

        # Stop if end token is predicted or max length reached
        if int_to_char[next_char_index] == end_token or len(generated_sequence) >= max_len:
            break

        # Append the predicted character index
        generated_sequence.append(next_char_index)

    # Convert sequence of indices back to characters (excluding start token)
    smiles = "".join([int_to_char[i] for i in generated_sequence[1:]])
    return smiles

generated_count = 0
valid_count = 0
attempts = 0
max_attempts = NUM_TO_GENERATE * 5 # Try more times to get valid ones

generated_molecules = []

while generated_count < NUM_TO_GENERATE and attempts < max_attempts:
    attempts += 1
    print(f"Attempt {attempts}...")
    new_smiles = generate_smiles(model, char_to_int, int_to_char, max_len_with_tokens)

    # Validate using RDKit
    mol = Chem.MolFromSmiles(new_smiles)
    if mol is not None:
        print(f"  Generated Valid SMILES: {new_smiles}")
        generated_molecules.append(new_smiles)
        valid_count += 1
        generated_count += 1
    else:
        # print(f"  Generated Invalid SMILES: {new_smiles}") # Optional: print invalid ones
        pass

print(f"\nFinished Generation.")
print(f"Successfully generated {valid_count} valid molecules out of {NUM_TO_GENERATE} requested (in {attempts} attempts).")
print("\nGenerated Valid Molecules:")
if generated_molecules:
    for i, smiles in enumerate(generated_molecules):
        print(f"{i+1}: {smiles}")
else:
    print("None")

print("\n--- Demonstration Finished ---")


Generating 10 new molecule attempts...
Attempt 1...


[06:58:22] SMILES Parse Error: extra close parentheses while parsing: CCOC(=O)C(=O)c1cc(O)c2nc3ccc(C(=O)O)c3c(=O)n2c(C(C)=O)C(=O)O)O2)C(=O)O1
[06:58:22] SMILES Parse Error: Failed parsing SMILES 'CCOC(=O)C(=O)c1cc(O)c2nc3ccc(C(=O)O)c3c(=O)n2c(C(C)=O)C(=O)O)O2)C(=O)O1' for input: 'CCOC(=O)C(=O)c1cc(O)c2nc3ccc(C(=O)O)c3c(=O)n2c(C(C)=O)C(=O)O)O2)C(=O)O1'


Attempt 2...


KeyboardInterrupt: 

In [ ]:
# Required libraries: deepchem, rdkit-pypi, tensorflow, numpy<2, tf_keras
# Install using pip in Colab (run in separate cell FIRST, then restart runtime):
# !pip install --upgrade pip
# !pip install "numpy<2.0.0"  # Install NumPy 1.x FIRST
# !pip install tf_keras # Install Keras 2 compatibility package
# import os; os.environ['TF_USE_LEGACY_KERAS'] = '1' # Set Env Var AFTER installing tf_keras
# !pip install --pre deepchem rdkit-pypi scikit-learn matplotlib tensorflow Pillow # Or torch

import deepchem as dc
import numpy as np
import tensorflow as tf
from tensorflow import keras
# Use Keras 2 API if using TF_USE_LEGACY_KERAS=1
# from tf_keras.layers import LSTM, Dense, Embedding, TimeDistributed
# from tf_keras.models import Sequential
# from tf_keras.preprocessing.sequence import pad_sequences
# from tf_keras.utils import to_categorical
# Or use standard Keras 3 if TF_USE_LEGACY_KERAS is not set or needed
from keras.layers import LSTM, Dense, Embedding, TimeDistributed
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from rdkit import Chem # For validating generated SMILES
import random

# --- Configuration ---
MAX_SMILES_LEN = 100 # Maximum length for SMILES strings (adjust based on dataset)
LSTM_UNITS = 128
EMBEDDING_DIM = 64
# *** MODIFICATION START ***
NUM_EPOCHS = 30 # Increased epochs for potentially better generation quality (was 5)
                # Note: This will significantly increase training time.
                # Even more epochs (50+) might be needed for higher validity rates.
# *** MODIFICATION END ***
BATCH_SIZE = 128
NUM_TO_GENERATE = 10 # Number of molecules to attempt generating

# --- 1. Load SMILES Data ---
print("Loading HIV dataset SMILES...")
# Provide a featurizer (e.g., 'ECFP') as required by molnet loader,
# even though we only need the SMILES IDs. Splitter is not needed.
tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', splitter=None)

# datasets is a tuple, get the first (and only) dataset
dataset = datasets[0]
all_smiles = dataset.ids # .ids usually holds the SMILES strings

# Filter SMILES by length
original_count = len(all_smiles)
all_smiles = [s for s in all_smiles if isinstance(s, str) and len(s) <= MAX_SMILES_LEN] # Add check for string type
print(f"Loaded {original_count} SMILES, using {len(all_smiles)} with length <= {MAX_SMILES_LEN}")

# --- 2. Preprocess SMILES Data ---
print("Preprocessing SMILES data...")

# Add start (!) and end (?) tokens
smiles_with_tokens = ['!' + s + '?' for s in all_smiles]

# Create character vocabulary
all_chars = set()
for s in smiles_with_tokens:
    all_chars.update(s)
chars = sorted(list(all_chars))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}
vocab_size = len(chars)
print(f"Vocabulary size: {vocab_size}")
print(f"Vocabulary: {''.join(chars)}")

# Convert SMILES to sequences of integers
sequences = [[char_to_int[c] for c in s] for s in smiles_with_tokens]

# Prepare input (X) and output (y) sequences for training
# X: sequence up to character t
# y: sequence shifted by one (predict character t+1)
X_seq, y_seq = [], []
for seq in sequences:
    # Ensure sequence has at least 2 characters (start + one char)
    if len(seq) > 1:
      # Input sequence (excluding the last character)
      X_seq.append(seq[:-1])
      # Output sequence (excluding the first character)
      y_seq.append(seq[1:])

print(f"Number of sequences for training: {len(X_seq)}")

# Pad sequences to ensure uniform length
# Use max length including start/end tokens
max_len_with_tokens = MAX_SMILES_LEN + 2
X_padded = pad_sequences(X_seq, maxlen=max_len_with_tokens, padding='post')
y_padded = pad_sequences(y_seq, maxlen=max_len_with_tokens, padding='post')

# One-hot encode the output sequences (y)
# Shape: (num_sequences, max_len_with_tokens, vocab_size)
y_one_hot = np.array([to_categorical(seq, num_classes=vocab_size) for seq in y_padded])

print(f"Shape of X_padded: {X_padded.shape}")
print(f"Shape of y_one_hot: {y_one_hot.shape}")


# --- 3. Define LSTM Model ---
print("Defining LSTM model...")

model = Sequential([
    # Embedding layer: Turns integer indices into dense vectors
    Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=max_len_with_tokens),
    # LSTM layer: Processes sequences, returns full sequence output
    LSTM(LSTM_UNITS, return_sequences=True),
    # TimeDistributed Dense layer: Applies a Dense layer to each time step
    # Output layer predicts probability distribution over the vocabulary for the next character
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


# --- 4. Train LSTM Model ---
# Note: Training can be slow, especially without a GPU.
# Increase epochs for better generation quality.
print(f"\nTraining LSTM model for {NUM_EPOCHS} epochs...")
history = model.fit(X_padded, y_one_hot, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1)
print("Training finished.")

# --- 5. Generate New Molecules ---
print(f"\nGenerating {NUM_TO_GENERATE} new molecule attempts...")

def generate_smiles(model, char_to_int, int_to_char, max_len, start_token='!', end_token='?'):
    """Generates a SMILES string using the trained LSTM model."""
    vocab_size = len(char_to_int)
    generated_sequence = [char_to_int[start_token]]

    while True:
        # Pad the current sequence
        padded_sequence = pad_sequences([generated_sequence], maxlen=max_len, padding='post')

        # Predict probabilities for the next character
        # Predict gives shape (1, current_len, vocab_size), take last prediction
        probabilities = model.predict(padded_sequence, verbose=0)[0, len(generated_sequence)-1, :]

        # Sample the next character index based on probabilities
        # Using np.random.choice introduces randomness
        # Add small epsilon to probabilities to avoid sum-to-zero issues if model predicts all zeros
        probabilities = probabilities + 1e-8
        probabilities = probabilities / np.sum(probabilities) # Re-normalize

        next_char_index = np.random.choice(range(vocab_size), p=probabilities)

        # Stop if end token is predicted or max length reached
        if int_to_char[next_char_index] == end_token or len(generated_sequence) >= max_len:
            break

        # Append the predicted character index
        generated_sequence.append(next_char_index)

    # Convert sequence of indices back to characters (excluding start token)
    smiles = "".join([int_to_char[i] for i in generated_sequence[1:]])
    return smiles

generated_count = 0
valid_count = 0
attempts = 0
max_attempts = NUM_TO_GENERATE * 10 # Increase attempts multiplier slightly for longer training

generated_molecules = []

while generated_count < NUM_TO_GENERATE and attempts < max_attempts:
    attempts += 1
    # print(f"Attempt {attempts}...") # Reduce print frequency for longer runs
    new_smiles = generate_smiles(model, char_to_int, int_to_char, max_len_with_tokens)

    # Validate using RDKit
    mol = Chem.MolFromSmiles(new_smiles)
    if mol is not None:
        print(f"Attempt {attempts}: Generated Valid SMILES: {new_smiles}")
        generated_molecules.append(new_smiles)
        valid_count += 1
        generated_count += 1
    else:
        if attempts % 10 == 0: # Print progress occasionally even for invalid attempts
             print(f"Attempt {attempts}: Invalid SMILES generated.")
        # print(f"  Generated Invalid SMILES: {new_smiles}") # Optional: print invalid ones
        pass

print(f"\nFinished Generation.")
print(f"Successfully generated {valid_count} valid molecules out of {NUM_TO_GENERATE} requested (in {attempts} attempts).")
print("\nGenerated Valid Molecules:")
if generated_molecules:
    for i, smiles in enumerate(generated_molecules):
        print(f"{i+1}: {smiles}")
else:
    print("None")

print("\n--- Demonstration Finished ---")


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Loading HIV dataset SMILES...


Streaming output truncated to the last 5000 lines.
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:09:36] DEPRECATION WARNING: please use MorganGenerator
[07:0

Loaded 41120 SMILES, using 39960 with length <= 100
Preprocessing SMILES data...
Vocabulary size: 58
Vocabulary: !#%()+-.0123456789=?ABCFGHIKLMNOPRSTUVWZ[]abcdeghilnoprstu
Number of sequences for training: 39960
Shape of X_padded: (39960, 102)
Shape of y_one_hot: (39960, 102, 58)
Defining LSTM model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


...

Attempt 6: Generated Valid SMILES: NC(=S)NC1C(=O)NC1c1ccccc1
Attempt 7: Generated Valid SMILES: CN(C)CSSCc1ccccc1
Attempt 8: Generated Valid SMILES: O=Cc1cc(-c2ccc3ccccc3c2O)c(CC(O)C(N)C(O)CO)nc1Nc1ccccc1
Attempt 9: Generated Valid SMILES: Nc1ccc(N2CC2c2ccccc2)cc1
Attempt 10: Generated Valid SMILES: CC(=O)Oc1cccc(C=Nc2ccc(NC(C)=O)cc2)c1
Attempt 11: Generated Valid SMILES: COC1=Cc2ccccc2C(=O)C1=Cc1ccc(Br)cc1
Attempt 12: Generated Valid SMILES: CC(C)c1ccc(S(=O)(=O)Nc2ccc(C)cc2)cc1


[07:15:29] Explicit valence for atom # 19 C, 5, is greater than permitted
[07:15:31] SMILES Parse Error: unclosed ring for input: 'COc1cc2c(cc1O)-c1ccc3ccccc13'
[07:15:33] SMILES Parse Error: unclosed ring for input: 'O=C(NCCN1CCCCC1)c1nc2ccccn2c2ncncn1'


Attempt 16: Generated Valid SMILES: COC(=O)C(C)(C)NC(=O)c1ccc(C(=O)c2ccccc2)cc1.O=C(O)C=CC(C)C


[07:15:41] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11


Attempt 18: Generated Valid SMILES: CCOP(=O)(OCC)Cc1ccc(Nc2nc3ccccc3s2)cc1

Finished Generation.
Successfully generated 10 valid molecules out of 10 requested (in 18 attempts).

Generated Valid Molecules:
1: CCOC(=O)C(C#N)=Cc1ccc(OC)c(OC)c1OC
2: NC(=S)NC1C(=O)NC1c1ccccc1
3: CN(C)CSSCc1ccccc1
4: O=Cc1cc(-c2ccc3ccccc3c2O)c(CC(O)C(N)C(O)CO)nc1Nc1ccccc1
5: Nc1ccc(N2CC2c2ccccc2)cc1
6: CC(=O)Oc1cccc(C=Nc2ccc(NC(C)=O)cc2)c1
7: COC1=Cc2ccccc2C(=O)C1=Cc1ccc(Br)cc1
8: CC(C)c1ccc(S(=O)(=O)Nc2ccc(C)cc2)cc1
9: COC(=O)C(C)(C)NC(=O)c1ccc(C(=O)c2ccccc2)cc1.O=C(O)C=CC(C)C
10: CCOP(=O)(OCC)Cc1ccc(Nc2nc3ccccc3s2)cc1

--- Demonstration Finished ---


In [ ]:
# Required libraries: rdkit-pypi
# Install using pip if needed:
# !pip install rdkit-pypi

import sys
import os
from rdkit import Chem
from rdkit.Chem import AllChem # Needed for 3D embedding and optimization

# --- User Input ---
# PASTE YOUR LIST OF SMILES STRINGS HERE
# Replace the example list below with the SMILES generated by the previous script
smiles_list = [
    "CCOC(=O)C(C#N)=Cc1ccc(OC)c(OC)c1OC",
    "NC(=S)NC1C(=O)NC1c1ccccc1",
    "CN(C)CSSCc1ccccc1",
    "O=Cc1cc(-c2ccc3ccccc3c2O)c(CC(O)C(N)C(O)CO)nc1Nc1ccccc1",
    "Nc1ccc(N2CC2c2ccccc2)cc1",
    "CC(=O)Oc1cccc(C=Nc2ccc(NC(C)=O)cc2)c1",
    "COC1=Cc2ccccc2C(=O)C1=Cc1ccc(Br)cc1",
    "CC(C)c1ccc(S(=O)(=O)Nc2ccc(C)cc2)cc1",
    "COC(=O)C(C)(C)NC(=O)c1ccc(C(=O)c2ccccc2)cc1.O=C(O)C=CC(C)C", # Example with salt/fragment
    "CCOP(=O)(OCC)Cc1ccc(Nc2nc3ccccc3s2)cc1"
    # Add more SMILES strings as needed
]

# --- Configuration ---
output_directory = "sdf_3d_structures" # Directory to save .sdf files
optimize_geometry = True # Set to False to skip force field optimization

# --- Processing ---
print(f"Processing {len(smiles_list)} SMILES strings for 3D structure generation...")

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created output directory: {output_directory}")

molecules_processed = 0
molecules_saved = 0
embedding_failures = 0
optimization_failures = 0 # Only relevant if optimize_geometry is True

for i, smiles in enumerate(smiles_list):
    mol_3d = None # Reset for each SMILES
    print(f"\nProcessing SMILES #{i+1}: {smiles}")
    if not isinstance(smiles, str):
        print(f"  Skipping: Item is not a string.")
        continue

    molecules_processed += 1

    # 1. Convert SMILES to Mol object
    mol_2d = Chem.MolFromSmiles(smiles)
    if mol_2d is None:
        print(f"  Skipping: Could not parse SMILES string.")
        continue

    # 2. Add Hydrogens (important for 3D structure)
    mol_h = Chem.AddHs(mol_2d)
    if mol_h is None:
        print(f"  Skipping: Could not add Hydrogens.")
        continue

    # 3. Embed 3D Coordinates
    # Tries to generate a 3D conformation. Returns an ID (-1 on failure).
    embed_status = AllChem.EmbedMolecule(mol_h, randomSeed=42) # Use a seed for reproducibility
    if embed_status == -1:
        print(f"  Skipping: Could not generate 3D coordinates (embedding failed).")
        embedding_failures += 1
        # Sometimes trying again with different parameters helps
        # embed_status = AllChem.EmbedMolecule(mol_h, useRandomCoords=True, randomSeed=42)
        # if embed_status == -1:
        #     print(f"  Skipping: Embedding failed even with random coords.")
        #     embedding_failures +=1
        #     continue
        # else:
        #      print("  Embedding succeeded with random coords.")
        continue # Skip if embedding failed
    else:
        print("  3D coordinates embedded successfully.")
        mol_3d = mol_h # Keep the molecule with 3D coords

    # 4. Optimize Geometry (Optional but Recommended)
    if optimize_geometry and mol_3d:
        print("  Optimizing geometry using MMFF94 force field...")
        try:
            opt_status = AllChem.MMFFOptimizeMolecule(mol_3d)
            # opt_status 0 means convergence, 1 means not converged, -1 means failure
            if opt_status == 0:
                print("  Geometry optimization converged.")
            elif opt_status == 1:
                print("  Warning: Geometry optimization did not fully converge.")
            else: # opt_status == -1 or other errors
                 print(f"  Warning: Geometry optimization failed (status: {opt_status}). Using unoptimized coordinates.")
                 optimization_failures += 1
        except Exception as e:
            # Catch potential exceptions during optimization
            print(f"  Warning: Geometry optimization threw an exception: {e}. Using unoptimized coordinates.")
            optimization_failures += 1
            # Keep mol_3d as the unoptimized version

    # 5. Save to SDF file
    if mol_3d:
        # Define output filename (e.g., mol_1.sdf, mol_2.sdf)
        output_filename = os.path.join(output_directory, f"mol_{i+1}.sdf")
        try:
            # Use SDWriter to save the molecule with 3D coords
            writer = Chem.SDWriter(output_filename)
            # Add name property if desired (using original SMILES here)
            mol_3d.SetProp("_Name", smiles)
            writer.write(mol_3d)
            writer.close()
            print(f"  Successfully saved 3D structure to {output_filename}")
            molecules_saved += 1
        except Exception as e:
            print(f"  ERROR: Could not save molecule to {output_filename}: {e}")

# --- Summary ---
print("\n--- 3D Structure Generation Finished ---")
print(f"Total SMILES processed: {molecules_processed}")
print(f"Successfully saved 3D structures (.sdf): {molecules_saved}")
if embedding_failures > 0:
    print(f"Failed to generate 3D coordinates (embedding): {embedding_failures}")
if optimize_geometry and optimization_failures > 0:
     print(f"Geometry optimization issues (failed or did not converge): {optimization_failures}")
print(f"Output files are located in the '{output_directory}' directory.")
print("You can open these .sdf files with molecular viewers like PyMOL, Chimera, Avogadro, etc.")



Processing 10 SMILES strings for 3D structure generation...
Created output directory: sdf_3d_structures

Processing SMILES #1: CCOC(=O)C(C#N)=Cc1ccc(OC)c(OC)c1OC
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Geometry optimization converged.
  Successfully saved 3D structure to sdf_3d_structures/mol_1.sdf

Processing SMILES #2: NC(=S)NC1C(=O)NC1c1ccccc1
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Geometry optimization converged.
  Successfully saved 3D structure to sdf_3d_structures/mol_2.sdf

Processing SMILES #3: CN(C)CSSCc1ccccc1
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Geometry optimization converged.
  Successfully saved 3D structure to sdf_3d_structures/mol_3.sdf

Processing SMILES #4: O=Cc1cc(-c2ccc3ccccc3c2O)c(CC(O)C(N)C(O)CO)nc1Nc1ccccc1
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Success

In [ ]:
!zip -r  sdf.zip sdf_3d_structures/

  adding: sdf_3d_structures/ (stored 0%)
  adding: sdf_3d_structures/mol_7.sdf (deflated 77%)
  adding: sdf_3d_structures/mol_4.sdf (deflated 78%)
  adding: sdf_3d_structures/mol_8.sdf (deflated 78%)
  adding: sdf_3d_structures/mol_9.sdf (deflated 78%)
  adding: sdf_3d_structures/mol_5.sdf (deflated 77%)
  adding: sdf_3d_structures/mol_6.sdf (deflated 77%)
  adding: sdf_3d_structures/mol_3.sdf (deflated 77%)
  adding: sdf_3d_structures/mol_1.sdf (deflated 77%)
  adding: sdf_3d_structures/mol_10.sdf (deflated 77%)
  adding: sdf_3d_structures/mol_2.sdf (deflated 76%)


In [ ]:
!pip install py3Dmol

In [ ]:
# Required libraries: rdkit-pypi, py3Dmol
# Install using pip if needed:
# !pip install rdkit-pypi py3Dmol

import sys
import os
from rdkit import Chem
from rdkit.Chem import AllChem # Needed for 3D embedding and optimization
import py3Dmol # For interactive 3D visualization in Colab/Jupyter

# --- User Input ---
# PASTE YOUR LIST OF SMILES STRINGS HERE
# Replace the example list below with the SMILES generated by the previous script
smiles_list = [
    "CCOC(=O)C(C#N)=Cc1ccc(OC)c(OC)c1OC",
    "NC(=S)NC1C(=O)NC1c1ccccc1",
    "CN(C)CSSCc1ccccc1",
    "O=Cc1cc(-c2ccc3ccccc3c2O)c(CC(O)C(N)C(O)CO)nc1Nc1ccccc1",
    "Nc1ccc(N2CC2c2ccccc2)cc1",
    "CC(=O)Oc1cccc(C=Nc2ccc(NC(C)=O)cc2)c1",
    "COC1=Cc2ccccc2C(=O)C1=Cc1ccc(Br)cc1",
    "CC(C)c1ccc(S(=O)(=O)Nc2ccc(C)cc2)cc1",
    "COC(=O)C(C)(C)NC(=O)c1ccc(C(=O)c2ccccc2)cc1.O=C(O)C=CC(C)C", # Example with salt/fragment
    "CCOP(=O)(OCC)Cc1ccc(Nc2nc3ccccc3s2)cc1"
    # Add more SMILES strings as needed
]

# --- Configuration ---
optimize_geometry = True # Set to False to skip force field optimization
viewer_width = 400
viewer_height = 300

# --- Processing ---
print(f"Processing {len(smiles_list)} SMILES strings for 3D visualization...")

molecules_processed = 0
molecules_visualized = 0
embedding_failures = 0
optimization_failures = 0 # Only relevant if optimize_geometry is True

for i, smiles in enumerate(smiles_list):
    mol_3d = None # Reset for each SMILES
    print(f"\nProcessing SMILES #{i+1}: {smiles}")
    if not isinstance(smiles, str):
        print(f"  Skipping: Item is not a string.")
        continue

    molecules_processed += 1

    # 1. Convert SMILES to Mol object
    mol_2d = Chem.MolFromSmiles(smiles)
    if mol_2d is None:
        print(f"  Skipping: Could not parse SMILES string.")
        continue

    # 2. Add Hydrogens (important for 3D structure)
    mol_h = Chem.AddHs(mol_2d)
    if mol_h is None:
        print(f"  Skipping: Could not add Hydrogens.")
        continue

    # 3. Embed 3D Coordinates
    embed_status = AllChem.EmbedMolecule(mol_h, randomSeed=42)
    if embed_status == -1:
        print(f"  Skipping: Could not generate 3D coordinates (embedding failed).")
        embedding_failures += 1
        continue
    else:
        print("  3D coordinates embedded successfully.")
        mol_3d = mol_h

    # 4. Optimize Geometry (Optional but Recommended)
    if optimize_geometry and mol_3d:
        print("  Optimizing geometry using MMFF94 force field...")
        try:
            opt_status = AllChem.MMFFOptimizeMolecule(mol_3d)
            if opt_status == 0:
                print("  Geometry optimization converged.")
            elif opt_status == 1:
                print("  Warning: Geometry optimization did not fully converge.")
            else:
                 print(f"  Warning: Geometry optimization failed (status: {opt_status}). Using unoptimized coordinates.")
                 optimization_failures += 1
        except Exception as e:
            print(f"  Warning: Geometry optimization threw an exception: {e}. Using unoptimized coordinates.")
            optimization_failures += 1

    # 5. Visualize using py3Dmol
    if mol_3d:
        try:
            # Convert RDKit Mol object to SDF format string
            # *** MODIFICATION START ***
            # Remove the incorrect 'keepConformer' argument.
            # MolToMolBlock includes the default conformer (-1) by default.
            sdf_string = Chem.MolToMolBlock(mol_3d)
            # *** MODIFICATION END ***

            # Create py3Dmol viewer
            view = py3Dmol.view(width=viewer_width, height=viewer_height)

            # Add molecule data (SDF format)
            view.addModel(sdf_string, 'sdf')

            # Set visualization style (e.g., stick, sphere, cartoon)
            view.setStyle({'stick': {}})
            # view.setStyle({'sphere': {'scale': 0.3}}) # Alternative style

            # Zoom to fit the molecule
            view.zoomTo()

            # Display the viewer in the notebook output cell
            print(f"  Displaying 3D structure for: {smiles}")
            view.show()
            molecules_visualized += 1

        except Exception as e:
            print(f"  ERROR: Could not visualize molecule #{i+1}: {e}")

# --- Summary ---
print("\n--- 3D Visualization Finished ---")
print(f"Total SMILES processed: {molecules_processed}")
print(f"Successfully generated and displayed 3D structures: {molecules_visualized}")
if embedding_failures > 0:
    print(f"Failed to generate 3D coordinates (embedding): {embedding_failures}")
if optimize_geometry and optimization_failures > 0:
     print(f"Geometry optimization issues (failed or did not converge): {optimization_failures}")
print("Interactive viewers should appear above in the output cell.")



Processing 10 SMILES strings for 3D visualization...

Processing SMILES #1: CCOC(=O)C(C#N)=Cc1ccc(OC)c(OC)c1OC
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Geometry optimization converged.
  Displaying 3D structure for: CCOC(=O)C(C#N)=Cc1ccc(OC)c(OC)c1OC


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


Processing SMILES #2: NC(=S)NC1C(=O)NC1c1ccccc1
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Geometry optimization converged.
  Displaying 3D structure for: NC(=S)NC1C(=O)NC1c1ccccc1


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


Processing SMILES #3: CN(C)CSSCc1ccccc1
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Geometry optimization converged.
  Displaying 3D structure for: CN(C)CSSCc1ccccc1


...


Processing SMILES #9: COC(=O)C(C)(C)NC(=O)c1ccc(C(=O)c2ccccc2)cc1.O=C(O)C=CC(C)C
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Geometry optimization converged.
  Displaying 3D structure for: COC(=O)C(C)(C)NC(=O)c1ccc(C(=O)c2ccccc2)cc1.O=C(O)C=CC(C)C


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


Processing SMILES #10: CCOP(=O)(OCC)Cc1ccc(Nc2nc3ccccc3s2)cc1
  3D coordinates embedded successfully.
  Optimizing geometry using MMFF94 force field...
  Displaying 3D structure for: CCOP(=O)(OCC)Cc1ccc(Nc2nc3ccccc3s2)cc1


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


--- 3D Visualization Finished ---
Total SMILES processed: 10
Successfully generated and displayed 3D structures: 10
Interactive viewers should appear above in the output cell.
